In [188]:
import pandas as pd

In [189]:
df = pd.read_csv('titanic/test.csv')

In [190]:
df.shape

(418, 11)

In [191]:
df.head

<bound method NDFrame.head of      PassengerId  Pclass                                          Name  \
0            892       3                              Kelly, Mr. James   
1            893       3              Wilkes, Mrs. James (Ellen Needs)   
2            894       2                     Myles, Mr. Thomas Francis   
3            895       3                              Wirz, Mr. Albert   
4            896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)   
..           ...     ...                                           ...   
413         1305       3                            Spector, Mr. Woolf   
414         1306       1                  Oliva y Ocana, Dona. Fermina   
415         1307       3                  Saether, Mr. Simon Sivertsen   
416         1308       3                           Ware, Mr. Frederick   
417         1309       3                      Peter, Master. Michael J   

        Sex   Age  SibSp  Parch              Ticket      Fare Cabin Embarked  
0 

In [192]:
## check for empty rows
print(df.isnull().sum())

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [193]:
print(df['Fare'].isnull().sum())

1


In [194]:
## combining siblings and parents to get family size rather than having two different variables
df['fam_size'] = df['Parch'] + df['SibSp']

In [195]:
print(df['fam_size'])

0      0
1      1
2      0
3      0
4      2
      ..
413    0
414    0
415    0
416    0
417    2
Name: fam_size, Length: 418, dtype: int64


In [196]:
print(df['Name'][10:20])

10                                     Ilieff, Mr. Ylio
11                           Jones, Mr. Charles Cresson
12        Snyder, Mrs. John Pillsbury (Nelle Stevenson)
13                                 Howard, Mr. Benjamin
14    Chaffee, Mrs. Herbert Fuller (Carrie Constance...
15        del Carlo, Mrs. Sebastiano (Argenia Genovesi)
16                                    Keane, Mr. Daniel
17                                    Assaf, Mr. Gerios
18                         Ilmakangas, Miss. Ida Livija
19                Assaf Khalil, Mrs. Mariana (Miriam")"
Name: Name, dtype: object


In [197]:
## I want to use the title as a variable so I split out the part of the string between ',' and '.'
df['comma'] = df['Name'].str.find(',') + 2
df['fullstop'] = df['Name'].str.find('.')

In [198]:
df['Title'] = df.apply(lambda x: x['Name'][x['comma']:x['fullstop']],axis=1)

In [199]:
print(df['Title'][700: 710])

Series([], Name: Title, dtype: object)


In [200]:
## the age column misses a lot of values 177/891 so the question what to do about it. 
## one approach would be to take the mean/average 
df['Age'].mean()

30.272590361445783

In [201]:
## Checking whether title can help here

In [202]:
set(df['Title'])

{'Col', 'Dona', 'Dr', 'Master', 'Miss', 'Mr', 'Mrs', 'Ms', 'Rev'}

In [203]:
## some of the titles are very obscure so replace them with 'Rare Title' I also want to consolidate Miss and Mlle etc.
df['New_Title'] = df['Title'].replace(['Capt', 'Col', 'Don', 'Dr', 'Jonkheer', 'Lady', 'Major', 'Rev', 'Sir', 'the Countess', 'Dona' ], 'Rare Title')

In [204]:
df['New_Title'] = df['New_Title'].replace(['Mlle', 'Ms'], 'Miss')
df['New_Title'] = df['New_Title'].replace(['Mme'], 'Mrs')
print(df['New_Title'][250: 260])

250    Miss
251      Mr
252      Mr
253      Mr
254      Mr
255      Mr
256      Mr
257      Mr
258    Miss
259      Mr
Name: New_Title, dtype: object


In [205]:
set(df['New_Title'])

{'Master', 'Miss', 'Mr', 'Mrs', 'Rare Title'}

In [206]:
## correlation between age and new_title
df['New_Title_Num'] = df['New_Title'].replace({'Master': 0,'Miss': 1, 'Mr': 2, 'Mrs': 3, 'Rare Title': 4})


In [207]:
df['Age'].corr(df['New_Title_Num'])

0.5321625468322725

In [208]:
## there is a fairly high correlation
df['New_Title'].value_counts()

Mr            240
Miss           79
Mrs            72
Master         21
Rare Title      6
Name: New_Title, dtype: int64

In [209]:
df.groupby('New_Title')['Age'].mean()

New_Title
Master         7.406471
Miss          21.774844
Mr            32.000000
Mrs           38.903226
Rare Title    43.833333
Name: Age, dtype: float64

In [210]:
## ok so apparently Master is a little boy..  So I want to fill in the missing 
## age values with the average ages of the corresponding title

In [211]:
df['Age_Average'] = df['Age']

In [212]:
df.loc[(df['New_Title'] == 'Master'), 'Age_Average'] = 4.57
df.loc[(df['New_Title'] == 'Miss'), 'Age_Average'] = 21.85
df.loc[(df['New_Title'] == 'Mr'), 'Age_Average'] = 32.36
df.loc[(df['New_Title'] == 'Mrs'), 'Age_Average'] = 35.79
df.loc[(df['New_Title'] == 'Rare Title'), 'Age_Average'] = 45.55

In [213]:
df['Complete_Age'] = df['Age']
df['Complete_Age'].fillna(df['Age_Average'],inplace = True)

In [214]:
print(df.isnull().sum())

PassengerId        0
Pclass             0
Name               0
Sex                0
Age               86
SibSp              0
Parch              0
Ticket             0
Fare               1
Cabin            327
Embarked           0
fam_size           0
comma              0
fullstop           0
Title              0
New_Title          0
New_Title_Num      0
Age_Average        0
Complete_Age       0
dtype: int64


In [215]:
df['Embarked'].fillna('S', inplace = True)

In [216]:
print(df.isnull().sum())

PassengerId        0
Pclass             0
Name               0
Sex                0
Age               86
SibSp              0
Parch              0
Ticket             0
Fare               1
Cabin            327
Embarked           0
fam_size           0
comma              0
fullstop           0
Title              0
New_Title          0
New_Title_Num      0
Age_Average        0
Complete_Age       0
dtype: int64


In [217]:
## We have now simplified the titles, created a family(size) variable and replaced the missing age values
## with meaningful values 
## I want to use Pclass, Sex, Complete_Age, Fare, fam_size and Embarked es explanatory variables
## Sex and Embarked are strings so should be turned into numbers or perhaps categories for analysis

In [218]:
## create new column true if male false if female
df['Sex-Bool'] = (df['Sex'] == 'male') 

In [219]:
## categorize Embarked
df = pd.concat([df, pd.get_dummies(df['Embarked'])], axis=1)

In [220]:
## categorize New_Title
df = pd.concat([df, pd.get_dummies(df['New_Title'])], axis=1)

In [221]:
df.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'fam_size', 'comma', 'fullstop',
       'Title', 'New_Title', 'New_Title_Num', 'Age_Average', 'Complete_Age',
       'Sex-Bool', 'C', 'Q', 'S', 'Master', 'Miss', 'Mr', 'Mrs', 'Rare Title'],
      dtype='object')

In [222]:
print(df['Fare'].mean());


35.627188489208635


In [223]:
df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
df['Fare'] = df['Fare'].round(2)

In [224]:
features_df = df[['Pclass', 'Fare', 'fam_size', 'Master', 'Miss', 'Mr',
       'Mrs', 'Rare Title']]

In [225]:
X = features_df.values

In [226]:
import pickle
model_tree_01 = pickle.load( open( "model_tree_01", "rb" ) )

In [227]:
model_tree_01_prediction = model_tree_01.predict(X)

In [228]:
df_submit_1 = pd.DataFrame()

In [229]:
df_submit_1['PassengerId'] = df['PassengerId']

In [230]:
df_submit_1['Survived'] = model_tree_01_prediction

In [231]:
df_submit_1.head

<bound method NDFrame.head of      PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         0
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]>

In [233]:
df_submit_1.to_csv('submit_1', index=False)

In [234]:
logreg_01 = pickle.load(open("logreg_01", "rb"))

In [237]:
features_df = df[['Pclass', 'Fare', 'fam_size', 'Master', 'Miss', 'Mr',
       'Mrs', 'Rare Title', 'Sex-Bool', 'C', 'Q', 'S', 'Complete_Age']]

In [239]:
X = features_df.values

In [240]:
logreg_01_prediction = logreg_01.predict(X)

In [241]:
df_submit_2 = pd.DataFrame()
df_submit_2['PassengerId'] = df['PassengerId']
df_submit_2['Survived'] = logreg_01_prediction

In [244]:
df_submit_2.to_csv('submit_2', index=False)